In [ ]:
#!pip install agentpy owlready2
#!pip install agentpy

In [ ]:
import agentpy as ap
from matplotlib import pyplot as plt
import IPython
import random
from owlready2 import *

In [ ]:
onto = get_ontology("file://onto.owl")

In [ ]:
#ONTOLOGIA
#onto.delete()
with onto:

  class Entity(Thing):
    pass

  class Robot(Entity):
    pass

  class Object(Entity):
    pass

  class Pile(Entity):
    pass

  class Place(Thing):
    pass

  class has_place(ObjectProperty, FunctionalProperty):
        domain = [Entity]
        range = [Place]

  class has_position(DataProperty, FunctionalProperty):
        domain = [Place]
        range = [str]

"""
  class carries_object(ObjectProperty, FunctionalProperty):
        domain = [Robot]
        range = [Object]

  class is_in_pile(ObjectProperty, FunctionalProperty):
        domain = [Object]
        range = [Pile]

  class stack_height(DataProperty, FunctionalProperty):
        domain = [Pile]
        range = [int]
"""

In [ ]:
class ObjectAgent(ap.Agent):
    def setup(self):
        self.agentType = 1
        self.first_step = True
        self.pos = None
        pass

    def step(self):
        if self.first_step:
          self.pos = self.model.grid.positions[self]
          self.first_step = False
        pass
    pass

In [ ]:
class PileAgent(ap.Agent): #Ver si la pila si estaría definida como agente
    def setup(self):
        self.agentType = 2
        self.height = 0
    pass

In [ ]:
class RobotAgent(ap.Agent):

    # RAZONAMIENTO

    def see(self, e):
        # Detectar el entorno cercano y obtener información sobre objetos y obstáculos.
        self.per = []
        vecinos = e.neighbors(self, 1)
        #print(vecinos.to_dlist())
        for vecino in vecinos:
            #print("Este es vecino: ", vecino, " de ", self)
            # Ejemplo de detección de objetos
            #if e.contains_object(vecino): #cHANCE ESTO SOLUCIONA EL TEMA DE QUE se colisionen y se paren, ver como lo escribimos bien
            #Mas que nada es la logica para ue sus veicnos estemos seguro que sean objetos (cajas) y no otros robots

            #Creo que tenemos que poner cuales son otros robots vs objetos?
            #print(e.positions[vecino])
            if isinstance(vecino, ObjectAgent):
                self.per.append(Object(has_place=Place(has_position=str(e.positions[vecino]))))
            elif isinstance(vecino, RobotAgent):
                self.per.append(Robot(has_place=Place(has_position=str(e.positions[vecino]))))
            elif isinstance(vecino, PileAgent):
                self.per.append(Pile(has_place=Place(has_position=str(e.positions[vecino]))))
            #print(self.per)

    def next(self):
        # Evaluar reglas y decidir la acción
        for act in self.actions:
            for rule in self.rules:
                if rule(act):
                    print(self.per, "El robot ", self, " hara la accion ", act, self.model.grid.positions[self])
                    act()
                    return
        print("No hay reglas para el robot ", self, " en la posición ", self.model.grid.positions[self], " con la percepción ", self.per)

    #REGLAS

    def rule_pick(self, act):

        validador = [False, False]

        # Proposición 1: Si hay una caja en la posición actual y no se está llevando otra
        for caja in self.model.objects:
            if caja.pos == self.model.grid.positions[self] and not self.carrying_object:
                print("Estoy en una caja ", self, " y no tengo nada" , self.carrying_object)
                validador[0] = True

        # Proposición 2: Si la acción es la de recoger una caja
        if act == self.pick:
            validador[1] = True

        return sum(validador) == 2


    def rule_drop(self, act):

        validador = [False, False]

        # Proposición 1: Si se está llevando una caja
        if self.carrying_object:
            validador[0] = True

        # Proposición 2: Si la acción es la de dejar una caja
        if act == self.drop:
            validador[1] = True

        return sum(validador) == 2

    def rule_2(self,act):
        """
        Regla deductiva para moverse hacia el norte
        @param act: acción a validar
        @return: booleano
        """

        # Validador de regla
        validador = [False,False]

        # Proposición 1: Si hay suciedad en la posición Norte
        for caja in self.per:
            if eval(caja.has_place.has_position)[0] == self.model.grid.positions[self][0]-1:
                validador[0] = True

        # Proposición 2: Si la acción es la de moverse hacia el norte
        if act == self.move_N:
            validador[1] = True

        return sum(validador) == 2

    def rule_3(self,act):
        """
        Regla deductiva para moverse hacia el sur
        @param act: acción a validar
        @return: booleano
        """

        # Validador de regla
        validador = [False,False]

        # Proposición 1: Si hay suciedad en la posición Sur
        for caja in self.per:
            if eval(caja.has_place.has_position)[0] == self.model.grid.positions[self][0]+1:
                validador[0] = True

        # Proposición 2: Si la acción es la de moverse hacia el sur
        if act == self.move_S:
            validador[1] = True

        return sum(validador) == 2


    def rule_4(self,act):
        """
        Regla deductiva para moverse hacia el este
        @param act: acción a validar
        @return: booleano
        """

        # Validador de regla
        validador = [False,False]

        # Proposición 1: Si hay suciedad en la posición Este
        for caja in self.per:
            if eval(caja.has_place.has_position)[1] == self.model.grid.positions[self][1]+1:
                validador[0] = True

        # Proposición 2: Si la acción es la de moverse hacia el este
        if act == self.move_E:
            validador[1] = True

        return sum(validador) == 2


    def rule_5(self,act):
        """
        Regla deductiva para moverse hacia el oeste
        @param act: acción a validar
        @return: booleano
        """

        # Validador de regla
        validador = [False,False]

        # Proposición 1: Si hay suciedad en la posición Oeste
        for caja in self.per:
            if eval(caja.has_place.has_position)[1] == self.model.grid.positions[self][1]-1:
                validador[0] = True

        # Proposición 2: Si la acción es la de moverse hacia el oeste
        if act == self.move_W:
            validador[1] = True

        return sum(validador) == 2


    def rule_6(self,act):
        """
        Regla deductiva para moverse aleatoriamente
        @param act: acción a validar
        @return: booleano
        """

        # Validador de regla
        validador = [False,False]

        # Proposición 1: Si no hay suciedad en el entorno
        if len(self.per) <= 0:
            validador[0] = True
        elif len(self.per) == 1 and self.per[0].has_place.has_position != self.model.grid.positions[self]:
            #print("Hay suciedad en el entorno", self.per[0].has_place.has_position)
            validador[0] = True

        # Proposición 2: Si la acción es la de moverse aleatoriamente
        if act == self.move_random:
            validador[1] = True

        return sum(validador) == 2


    # ACCIONES

    def pick(self):
        """
        Función para recoger una caja.
        """
        # Si hay una caja en la posición actual y no se está cargando otra caja
        for caja in self.model.grid.agents:
            if caja.agentType == 1:
                if caja.pos == self.model.grid.positions[self] and not self.carrying_object:
                    self.carrying_object = caja  # Asignar la caja al robot
                    self.model.grid.remove_agents(caja)  # Remover la caja de la celda
                    break  # Romper ciclo
                elif caja.pos == self.model.grid.positions[self] and self.carrying_object:
                    print("Ya tengo una caja")
                    break
        pass


    def drop(self):
        """
        Función para dejar una caja.
        """
        # Si se está llevando una caja
        if self.carrying_object:
            pos = self.model.grid.positions[self]
            pila = self.model.grid.get_pile(pos)

            if pila and pila.height < 5:
                pila.add(self.carrying_object)  # Añadir caja a la pila existente
            #Ver otras condiciones por si la pila esta llena o de plano no es pila

                self.carrying_object = None  # Ya no lleva ninguna caja, ppor el momento dentro del if
        pass


    def move_N(self):
        """
        Función de movimiento hacia el norte
        """
        self.direction = (-1,0) #Cambio de dirección
        self.forward() # Caminar un paso hacia adelante

    def move_S(self):
        """
        Función de movimiento hacia el sur
        """
        self.direction = (1,0) #Cambio de dirección
        self.forward() # Caminar un paso hacia adelante

    def move_E(self):
        """
        Función de movimiento hacia el este
        """
        self.direction = (0,1) #Cambio de dirección
        self.forward() # Caminar un paso hacia adelante

    def move_W(self):
        """
        Función de movimiento hacia el oeste
        """
        self.direction = (0,-1) #Cambio de dirección
        self.forward() # Caminar un paso hacia adelante

    def move_random(self):
        """
        Función de movimiento aleatorio
        """
        # Rotaciones aleatorias
        for _ in range(random.randint(0,4)):
            self.turn()
        self.forward() # Caminar un paso hacia adelante

    def forward(self):
        """
        Función de movimiento
        """
        self.model.grid.move_by(self, self.direction)
        pass

    def turn(self):
        """
        Función de rotación
        """
        if self.direction == (-1,0):
            self.direction = (0,1) #Hacia Este
        elif self.direction == (0,1):
            self.direction = (1,0)  #Hacia Sur
        elif self.direction == (1,0):
            self.direction = (0,-1) #Hacia Oeste
        elif self.direction == (0,-1):
            self.direction = (-1,0) #Hacia Norte
        pass

    #SIMULACIÓN DE AGENTE

    def setup(self):
        """
        Función de inicialización
        """

        self.agentType = 0 # Tipo de agente
        self.direction = (-1,0) # Dirección inicial
        self.carrying_object = False
        # Acciones del agente
        self.actions = (
            self.pick,
            self.move_N,
            self.move_S,
            self.move_E,
            self.move_W,
            self.move_random
        )
        # Reglas del agente
        self.rules = (
            self.rule_pick,
            #self.rule_drop,
            self.rule_2,
            self.rule_3,
            self.rule_4,
            self.rule_5,
            self.rule_6
        )
        pass

    def step(self):
        """
        Función paso a paso
        """
        self.see(self.model.grid) #Percepción del Grid
        self.next() #Razonamiento y acción
        pass

    def update(self):
        pass

    def end(self):
        pass


In [ ]:
class WarehouseModel(ap.Model):

    def setup(self):
        self.robots = ap.AgentList(self, self.p.R, RobotAgent)
        self.objects = ap.AgentList(self, self.p.K, ObjectAgent)  # K es el número total de objetos
        #self.piles = []

        self.grid = ap.Grid(self, (self.p.M, self.p.N), track_empty=True)  # M y N son las dimensiones de la cuadrícula

        self.grid.add_agents(self.robots, random=True, empty=True)
        self.grid.add_agents(self.objects, random=True, empty=True)

    def step(self):
        self.robots.step()
        self.objects.step()

    def update(self):
        # Aquí podrías recopilar datos sobre el estado de la simulación, por ejemplo, el número de movimientos.
        pass

    def end(self):
        # Calcular estadísticas al final de la simulación, como tiempo total y movimientos realizados.
        pass



In [ ]:
parameters = {
    'R' : 1,
    'K' : 5,
    'M' : 6,
    'N' : 6,
    'steps' : 10
}

In [ ]:
#A FUNCTION TO ANIMATE THEE SIMULATION

def animation_plot(model, ax):
    """
    Función de animación
    @param model: modelo
    @param ax: axes (matplotlib)
    """
    # Definición de atributo para tipo de agente
    agent_type_grid = model.grid.attr_grid('agentType')
    # Definición de gráfico con colores (de acuerdo al tipo de agente)
    ap.gridplot(agent_type_grid, cmap='Accent', ax=ax)
    # Definición de título del gráfico
    ax.set_title(f"Vaccum Model \n Time-step: {model.t}, "
                 f"Drity: {0}")

In [ ]:
#SIMULATION:

#Create figure (from matplotlib)
fig, ax = plt.subplots()

#Create model
model = WarehouseModel(parameters)

#Run with animation
#If you want to run it without animation then use instead:
#model.run()
animation = ap.animate(model, fig, ax, animation_plot)
#This step may take a while before you can see anything

#Print the final animation
IPython.display.HTML(animation.to_jshtml())

In [ ]:
#model = WarehouseModel(parameters)
#model.run()